#Devloped By: KARTIK KUMAR
#USN: 1CD22CS061

In [ ]:
# STEP 1: SETUP & INSTALLATIONS
!pip install -q transformers moviepy scenedetect git+https://github.com/openai/whisper.git ffmpeg-python nltk fuzzywuzzy python-Levenshtein

import os
import tempfile
from moviepy.editor import VideoFileClip, concatenate_videoclips
import whisper
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.tokenize import sent_tokenize
from fuzzywuzzy import fuzz
import nltk
nltk.download('punkt')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# STEP 2: LOAD INPUT VIDEO
video_path = "/content/input/test.mp4"  # Upload your video here
assert os.path.exists(video_path), "Video not found! Please upload to /content/."
video_clip = VideoFileClip(video_path)

In [ ]:
# STEP 3: SHOT DETECTION USING PYSCENEDETECT
video_manager = VideoManager([video_path])
scene_manager = SceneManager()
scene_manager.add_detector(ContentDetector(threshold=30.0))
video_manager.set_downscale_factor()
video_manager.start()
scene_manager.detect_scenes(frame_source=video_manager)
scene_list = scene_manager.get_scene_list()
print(f"Detected {len(scene_list)} scenes.")

ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 25.000 FPS, resolution: 1280 x 720
INFO:pyscenedetect:Detecting scenes...


Detected 99 scenes.


In [ ]:
# STEP 4: DEMUX AUDIO FROM EACH SHOT - WILL GIVE THE TRANSCRIPT FILE.
shots = []
asr_model = whisper.load_model("medium")

for i, (start, end) in enumerate(scene_list):
    shot_clip = video_clip.subclip(start.get_seconds(), end.get_seconds())
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        audio_path = tmp.name
    shot_clip.audio.write_audiofile(audio_path, fps=16000, nbytes=2, codec="pcm_s16le", verbose=False, logger=None)

    result = asr_model.transcribe(
        audio_path,
        language="en",
        word_timestamps=True,
        condition_on_previous_text=False
    )

    transcript = result['text'].strip()
    os.remove(audio_path)

    skip_phrases = ["thank you", "thanks for watching", "you", ""]
    if transcript.lower() in skip_phrases:
        continue

    shots.append({
        "index": i,
        "start_time": start.get_seconds(),
        "end_time": end.get_seconds(),
        "text": transcript,
        "clip": shot_clip
    })

print("✅ Audio transcription complete for all shots.")

# Transcription to /content/TRANSCRIBE.TXT
with open("/content/TRANSCRIBE.TXT", "w", encoding="utf-8") as f:
    for shot in shots:
        f.write(f"[{shot['start_time']} – {shot['end_time']}]:\n{shot['text']}\n\n")

print("Transcriptions saved to /content/TRANSCRIBE.TXT")


100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 54.5MiB/s]


✅ Audio transcription complete for all shots.
Transcriptions saved to /content/TRANSCRIBE.TXT


In [ ]:
# 📘 STEP 5: BART SUMMARIZATION WITH SMART FRAMING

from transformers import pipeline
import torch

# 🔄 Load summarizer
device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
    device=device
)

def frame_summary(summary):
    explanation_keywords = ['explain', 'demonstrate', 'describe', 'teach', 'how to', 'talk about', 'share', 'discuss']
    if any(kw in summary.lower() for kw in explanation_keywords):
        if not summary.lower().startswith("the speaker"):
            return f"The speaker explains that {summary[0].lower() + summary[1:]}"
    return summary

with open("/content/SUMMARIZED_TEXT.TXT", "w", encoding="utf-8") as f:
    for shot in shots:
        text = shot['text'].strip()
        if len(text.split()) < 25:
            summary = text
        else:

            draft = summarizer(
                text,
                max_length=60,
                min_length=20,
                length_penalty=2.5,
                no_repeat_ngram_size=3,
                do_sample=False
            )[0]['summary_text']


            summary = summarizer(
                draft,
                max_length=20,
                min_length=5,
                length_penalty=3.0,
                no_repeat_ngram_size=3,
                do_sample=False
            )[0]['summary_text']

        # Apply conditional framing
        final_summary = frame_summary(summary.strip())
        shot['summary'] = final_summary
        f.write(f"[{shot['start_time']}–{shot['end_time']}]: {final_summary}\n")

print("✅ STEP 5 complete: Smart summaries saved to /content/SUMMARIZED_TEXT.TXT")


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Your max_length is set to 60, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)
Your max_length is set to 60, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 60, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your max_length is set to 60, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...'

✅ STEP 5 complete: Smart summaries saved to /content/SUMMARIZED_TEXT.TXT
